In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

for dirname, _, filenames in os.walk("./"):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
        
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

./combined.csv
./weirdos2.csv
./weirdos.csv
./greenwich_village_housing_data.csv
./Project_2.ipynb
./combined2.csv
./.ipynb_checkpoints/combined2-checkpoint.csv
./.ipynb_checkpoints/Project_2-checkpoint.ipynb
./.ipynb_checkpoints/test-checkpoint.csv
./.ipynb_checkpoints/for_rent_homes-checkpoint.csv
./.ipynb_checkpoints/web_scraper-checkpoint.ipynb
./.ipynb_checkpoints/weirdos-checkpoint.csv
./.ipynb_checkpoints/combined-checkpoint.csv
./.ipynb_checkpoints/greenwich_village_housing_data-checkpoint.csv
./.ipynb_checkpoints/weirdos2-checkpoint.csv
./csv_files/combined.csv
./csv_files/for_rent_homes.csv
./csv_files/sold_homes.csv
./csv_files/pending_homes.csv
./csv_files/for_sale_homes.csv
./csv_files/.ipynb_checkpoints/for_sale_homes-checkpoint.csv
./csv_files/.ipynb_checkpoints/for_rent_homes-checkpoint.csv
./csv_files/.ipynb_checkpoints/sold_homes-checkpoint.csv
./csv_files/.ipynb_checkpoints/pending_homes-checkpoint.csv


In [98]:
import requests
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from homeharvest import scrape_property
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_absolute_percentage_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV


# 1. Frame the problem
Using the customer description, Define the problem your trying to solve in your own words (remember this is not technial but must be specific so the customer understands the project

Build a model (or a system of models) that accurately predicts housing prices for properties located within the Greenwich Village area in New York City. This system will take in a set of parameters (yet to be fully determined; could consist of the number of rooms, square feet, proximity to parks, etc.), and generate a realistic price for the parameters given. 

# 2. Get the Data 
Define how you recieved the data (provided, gathered..)

In [3]:
all_properties = [] 

locations = [
    '10003',
    '10011',
    '10014',
    '10012',
] # greenwich village zip codes 

listing_types = [
    'sold',
    'for_sale',
    'pending',
    'for_rent'
] # all listings

for loc in locations:
    for listing_type in listing_types:
        properties = scrape_property(
            location=loc,
            listing_type=listing_type,
            past_days=730
        )
        
        print(f'for zip code {loc} + listing type as {listing_type}, {len(properties)} results were found')
        
        all_properties.append(properties)
        # time.sleep(5)

all_properties = pd.concat(all_properties, ignore_index=True)
all_properties = all_properties.drop_duplicates(subset=['property_id'])
all_properties.to_csv('greenwich_village_housing_data.csv', index=False)

for zip code 10003 + listing type as sold, 540 results were found
for zip code 10003 + listing type as for_sale, 364 results were found
for zip code 10003 + listing type as pending, 74 results were found
for zip code 10003 + listing type as for_rent, 272 results were found
for zip code 10011 + listing type as sold, 760 results were found
for zip code 10011 + listing type as for_sale, 490 results were found
for zip code 10011 + listing type as pending, 69 results were found
for zip code 10011 + listing type as for_rent, 410 results were found
for zip code 10014 + listing type as sold, 376 results were found
for zip code 10014 + listing type as for_sale, 211 results were found
for zip code 10014 + listing type as pending, 29 results were found
for zip code 10014 + listing type as for_rent, 314 results were found
for zip code 10012 + listing type as sold, 168 results were found
for zip code 10012 + listing type as for_sale, 158 results were found
for zip code 10012 + listing type as pendi

/tmp/ipykernel_3507430/2261856486.py:30: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_properties = pd.concat(all_properties, ignore_index=True)


In [4]:
print(len(all_properties)) # result: 4088 properties
all_properties.head() # just checking

4113


,property_url,property_id,listing_id,permalink,mls,mls_id,status,mls_status,text,style,...,builder_id,builder_name,office_id,office_mls_set,office_name,office_email,office_phones,nearby_schools,primary_photo,alt_photos
0,https://www.realtor.com/realestateandhomes-det...,4981939294,<NA>,201-E-17th-St-Apt-4D_New-York_NY_10003_M49819-...,<NA>,<NA>,SOLD,<NA>,<NA>,CONDO,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,"New York City Geographic District # 2, SUCCESS...",<NA>,<NA>
1,https://www.realtor.com/realestateandhomes-det...,3069156911,<NA>,16-E-12th-St-C7_New-York_NY_10003_M30691-56911,<NA>,<NA>,SOLD,<NA>,<NA>,CONDO,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,"New York City Geographic District # 2, SUCCESS...",<NA>,<NA>
2,https://www.realtor.com/realestateandhomes-det...,4551301491,<NA>,111-4th-Ave-Apt-9M_New-York_NY_10003_M45513-01491,<NA>,<NA>,SOLD,<NA>,<NA>,COOP,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,"SUCCESS ACADEMY CHARTER SCHOOL-UNION SQUARE, N...",https://ap.rdcpix.com/3570669836/7314276e0210e...,https://ap.rdcpix.com/3570669836/7314276e0210e...
3,https://www.realtor.com/realestateandhomes-det...,9593591942,<NA>,11-5th-Ave_New-York_NY_10003_M95935-91942,<NA>,<NA>,SOLD,<NA>,<NA>,CONDO,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,"New York City Geographic District # 2, SUCCESS...",<NA>,<NA>
4,https://www.realtor.com/realestateandhomes-det...,9652341568,2980720988,201-E-17th-St-Apt-15J_New-York_NY_10003_M96523...,DENY,23396617,SOLD,Sold,Sponsor Sale - No Board Approval! This newly u...,COOP,...,<NA>,<NA>,2277890,O-DENY-77,111 Fifth,lead.manager@elliman.com,"[{'number': '2126454040', 'type': 'Office', 'p...","New York City Geographic District # 2, SUCCESS...",https://ap.rdcpix.com/68426f55b044b4439752b0df...,https://ap.rdcpix.com/68426f55b044b4439752b0df...


# 3. Explore the Data
Gain insights into the data you have from step 2, making sure to identify any bias

In [5]:
print(all_properties['zip_code'].value_counts())

zip_code
10011    1615
10003    1160
10014     877
10012     461
Name: count, dtype: int64


In [6]:
print(all_properties['status'].value_counts())

status
SOLD          1844
FOR_RENT      1031
FOR_SALE       868
PENDING        364
CONTINGENT       6
Name: count, dtype: int64


In [7]:
price_cols = [
    'list_price', 
    # 'list_price_min', 
    # 'list_price_max', 
    'sold_price', 
    'last_sold_price', 
    'assessed_value', 
    'estimated_value'
]

for col in price_cols:
    print(f'------DATA FOR COLUMN \'{col.upper()}\'------')
    
    null_counts = all_properties.groupby('status')[col].apply(lambda x: x.isnull().sum())
    for status in listing_types:
        total = len(all_properties[all_properties['status'] == status.upper()])
        if status.upper() in null_counts.index:
            print(f'number of null values ({status}): {null_counts[status.upper()]}/{total}')
        else: print(f'number of null values ({status}): 0')
            

    print(f'\nmin: {all_properties[col].min():,.2f}')
    print(f'max: {all_properties[col].max():,.2f}')
    print(f'q1: {all_properties[col].quantile(0.25):,.2f}')
    print(f'q3: {all_properties[col].quantile(0.75):,.2f}')
    print(f'mean: {all_properties[col].mean():,.2f}')
    print(f'median: {all_properties[col].median():,.2f}\n')

------DATA FOR COLUMN 'LIST_PRICE'------
number of null values (sold): 1263/1844
number of null values (for_sale): 1/868
number of null values (pending): 0/364
number of null values (for_rent): 116/1031

min: 1,535.00
max: 399,500,000.00
q1: 7,000.00
q3: 2,350,000.00
mean: 2,457,423.57
median: 849,000.00

------DATA FOR COLUMN 'SOLD_PRICE'------
number of null values (sold): 0/1844
number of null values (for_sale): 423/868
number of null values (pending): 183/364
number of null values (for_rent): 891/1031

min: 5,350.00
max: 138,930,068.00
q1: 800,000.00
q3: 3,000,000.00
mean: 2,878,524.92
median: 1,550,000.00

------DATA FOR COLUMN 'LAST_SOLD_PRICE'------
number of null values (sold): 0/1844
number of null values (for_sale): 423/868
number of null values (pending): 183/364
number of null values (for_rent): 891/1031

min: 5,350.00
max: 138,930,068.00
q1: 800,000.00
q3: 3,000,000.00
mean: 2,878,524.92
median: 1,550,000.00

------DATA FOR COLUMN 'ASSESSED_VALUE'------
number of null valu

## conclusion:
- use LIST_PRICE for all FOR_SALE and PENDING properties
- use SOLD_PRICE for all SOLD properties
- use LIST_PRICE for all FOR_RENT properties
    - for FOR_RENT properties missing the LIST_PRICE column:
        - average out their LIST_PRICE_MIN and LIST_PRICE_MAX
    - everything else just drop!

In [8]:
# analyze suspiciously low "sold prices" and other rows that r missing data

min_price_range = 200000

weirdos = all_properties[all_properties['sold_price'] < min_price_range]
weirdos.to_csv('weirdos.csv', index=False)
print(weirdos.head())

weirdos2 = all_properties[(all_properties['status'] == 'FOR_RENT') & (all_properties['list_price'].isnull())]
weirdos2.to_csv('weirdos2.csv', index=False)
print(weirdos2.head())

bed_null_count = len(all_properties[all_properties['beds'].isnull()])
bath_null_count = len(all_properties[all_properties['full_baths'].isnull()])
print(f'number of rows that have null values for beds column: {bed_null_count}')
print(f'number of rows that have null values for baths column: {bath_null_count}')

garage_nonnull_count = len(all_properties) - len(all_properties[all_properties['parking_garage'].isnull()])
print(f'number of rows that have a value for parking garage: {garage_nonnull_count}')

school_nonnull_count = len(all_properties) - len(all_properties[all_properties['nearby_schools'].isnull()])
print(f'number of rows that have a value for nearby schools: {school_nonnull_count}')

                                           property_url property_id  \
293   https://www.realtor.com/realestateandhomes-det...  9412159427   
420   https://www.realtor.com/realestateandhomes-det...  9387879318   
454   https://www.realtor.com/realestateandhomes-det...  9104456986   
671   https://www.realtor.com/realestateandhomes-det...  3168024681   
1274  https://www.realtor.com/realestateandhomes-det...  3823323900   

      listing_id                                          permalink   mls  \
293   2981857475  300-Mercer-St-Apt-10I_New-York_NY_10003_M94121...  DENY   
420         <NA>   70-E-10th-St-Hlwy_New-York_NY_10003_M93878-79318  <NA>   
454         <NA>  215-E-19th-St-Apt-11G_New-York_NY_10003_M91044...  <NA>   
671   2984902424       135-E-19th-St_New-York_NY_10003_M31680-24681  DENY   
1274        <NA>  351-W-24th-St-Apt-2H_New-York_NY_10011_M38233-...  <NA>   

        mls_id    status mls_status  \
293   23223350      SOLD       Sold   
420       <NA>      SOLD       <

In [9]:
print('all column names: ')
for col in all_properties:
    print(col)

all column names: 
property_url
property_id
listing_id
permalink
mls
mls_id
status
mls_status
text
style
formatted_address
full_street_line
street
unit
city
state
zip_code
beds
full_baths
half_baths
sqft
year_built
days_on_mls
list_price
list_price_min
list_price_max
list_date
pending_date
sold_price
last_sold_date
last_sold_price
assessed_value
estimated_value
tax
tax_history
new_construction
lot_sqft
price_per_sqft
latitude
longitude
neighborhoods
county
fips_code
stories
hoa_fee
parking_garage
agent_id
agent_name
agent_email
agent_phones
agent_mls_set
agent_nrds_id
broker_id
broker_name
builder_id
builder_name
office_id
office_mls_set
office_name
office_email
office_phones
nearby_schools
primary_photo
alt_photos


In [10]:
# further data analysis 

sold_df = all_properties[all_properties['status'] == 'SOLD']
for_rent_df = all_properties[all_properties['status'] == 'FOR_RENT']
for_sale_df = all_properties[all_properties['status'] == 'FOR_SALE']
pending_df = all_properties[all_properties['status'] == 'PENDING'] 

temp_dfs = [sold_df, for_rent_df, for_sale_df, pending_df] 

for df in temp_dfs: 
    print(f"\n----DATA ANALYSIS ON {df['status'].iloc[0]} DF----\n")
    print(f"PRICE RANGE:\n-----")
    print(f"min: {df['sold_price'].min():,.1f}\t\tmax: {df['sold_price'].max():,.1f}")
    print(f"median: {df['sold_price'].median():,.1f}\tmean: {df['sold_price'].mean():,.1f}")
    print(f"\nBED/BATH COLUMNS:\n-----")
    print(f"null bed rows: {len(df[df['beds'].isnull()])}/{len(df)}", end='\t\t')
    print(f"null bath rows: {len(df[df['full_baths'].isnull()])}/{len(df)}", end='\t')
    print(f"\nbeds count median: {df['beds'].median():.1f}\t\tbaths count median: {df['full_baths'].median():.1f}")
    print(f"beds count mean: {df['beds'].mean():.1f}\t\tbaths count mean: {df['full_baths'].mean():.1f}")


----DATA ANALYSIS ON SOLD DF----

PRICE RANGE:
-----
min: 5,350.0		max: 132,000,000.0
median: 1,800,000.0	mean: 3,181,807.1

BED/BATH COLUMNS:
-----
null bed rows: 1259/1844		null bath rows: 1781/1844	
beds count median: 1.0		baths count median: 1.0
beds count mean: 1.7		baths count mean: 1.4

----DATA ANALYSIS ON FOR_RENT DF----

PRICE RANGE:
-----
min: 139,250.0		max: 138,930,068.0
median: 1,099,855.0	mean: 2,606,568.0

BED/BATH COLUMNS:
-----
null bed rows: 115/1031		null bath rows: 119/1031	
beds count median: 1.0		baths count median: 1.0
beds count mean: 1.2		baths count mean: 1.2

----DATA ANALYSIS ON FOR_SALE DF----

PRICE RANGE:
-----
min: 22,100.0		max: 45,000,000.0
median: 1,200,000.0	mean: 2,232,050.1

BED/BATH COLUMNS:
-----
null bed rows: 0/868		null bath rows: 6/868	
beds count median: 2.0		baths count median: 2.0
beds count mean: 2.0		baths count mean: 1.9

----DATA ANALYSIS ON PENDING DF----

PRICE RANGE:
-----
min: 16,667.0		max: 15,000,000.0
median: 932,500.0	mean: 1

In [11]:
print(all_properties['nearby_schools'].value_counts())

nearby_schools
New York City Geographic District # 2                                                                                                                                     1957
New York City Geographic District # 2, New York City Special Schools - District 75                                                                                         724
New York City Geographic District # 2, SUCCESS ACADEMY CHARTER SCHOOL-UNION SQUARE                                                                                         572
New York City Geographic District # 2, SUCCESS ACADEMY CHARTER SCHOOL-UNION SQUARE, New York City Special Schools - District 75                                            182
SUCCESS ACADEMY CHARTER SCHOOL-UNION SQUARE, New York City Special Schools - District 75, New York City Geographic District # 1                                            157
New York City Special Schools - District 75, New York City Geographic District # 1                            

# 4.Prepare the Data


Apply any data transformations and explain what and why

In [12]:
# step 1: keep only relevant columns
keep_colnames = ['status',
                 'style',
                 'zip_code',
                 'beds',
                 'full_baths',
                 'half_baths',
                 'sqft',
                 'lot_sqft',
                 'year_built',
                 'list_price',
                 'list_price_min',
                 'list_price_max',
                 'sold_price',
                 'last_sold_date',
                 'new_construction',
                 'price_per_sqft',
                 'latitude',
                 'longitude',
                 'neighborhoods',
                 'stories',
                 'hoa_fee',
                 'nearby_schools'
                ]

new_df = all_properties[keep_colnames].copy()

# separate into sold, for_rent, pending, and for_sale

sold_homes = new_df[new_df['status'] == 'SOLD'].copy()
for_rent_homes = new_df[new_df['status'] == 'FOR_RENT'].copy()
pending_homes = new_df[new_df['status'] == 'PENDING'].copy()
for_sale_homes = new_df[new_df['status'] == 'FOR_SALE'].copy()

# fix list_price column null values in for_rent_homes

for_rent_homes['list_price'].fillna(
    for_rent_homes[['list_price_min', 'list_price_max']].mean(axis=1), inplace=True
) 

# CHECK IF EACH DF HAS A COMPLETELY-FILLED PRICE COLUMN

print(f"number of null SOLD_PRICE values in sold_homes: {len(sold_homes[sold_homes['sold_price'].isnull()])}")
print(f"number of null LIST_PRICE values in pending_homes: {len(pending_homes[pending_homes['list_price'].isnull()])}")
print(f"number of null LIST_PRICE values in for_sale_homes: {len(for_sale_homes[for_sale_homes['list_price'].isnull()])}")
print(f"number of null LIST_PRICE values in for_rent_homes: {len(for_rent_homes[for_rent_homes['list_price'].isnull()])}")

number of null SOLD_PRICE values in sold_homes: 0
number of null LIST_PRICE values in pending_homes: 0
number of null LIST_PRICE values in for_sale_homes: 1
number of null LIST_PRICE values in for_rent_homes: 1


/tmp/ipykernel_3507430/4030557409.py:37: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  for_rent_homes['list_price'].fillna(
/tmp/ipykernel_3507430/4030557409.py:37: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  for_rent_homes['list_price'].fillna(


In [13]:
# drop any final null rows (for_sale_homes and for_rent_homes) 

for_sale_homes.dropna(subset=['list_price'], axis=0, inplace=True)
for_rent_homes.dropna(subset=['list_price'], axis=0, inplace=True) 

# recheck 

print(f"number of null LIST_PRICE values in for_sale_homes: {len(for_sale_homes[for_sale_homes['list_price'].isnull()])}")
print(f"number of null LIST_PRICE values in for_rent_homes: {len(for_rent_homes[for_rent_homes['list_price'].isnull()])}\n")

for_sale_homes.to_csv('csv_files/for_sale_homes.csv', index=False)
sold_homes.to_csv('csv_files/sold_homes.csv', index=False)
pending_homes.to_csv('csv_files/pending_homes.csv', index=False)
for_rent_homes.to_csv('csv_files/for_rent_homes.csv', index=False)

pd.concat([for_sale_homes, sold_homes, pending_homes, for_rent_homes]).to_csv('csv_files/combined.csv', index=False)

number of null LIST_PRICE values in for_sale_homes: 0
number of null LIST_PRICE values in for_rent_homes: 0



In [14]:
# fill missing list_price values with sold_price
combined = pd.read_csv('csv_files/combined.csv')
mask = (combined['status'] == 'SOLD') & (combined['list_price'].isna())
combined.loc[mask, 'list_price'] = combined.loc[mask, 'sold_price']

# drop list_price_min and list_price_max
combined.drop(columns=['list_price_min', 'list_price_max'], inplace=True)

# for properties with stories = 0, just make the value nan
combined['stories'] = combined['stories'].replace(0, pd.NA)

# use median imputer for stories
stories_median = combined['stories'].median()
combined['stories'].fillna(stories_median, inplace=True)

# for properties with hoa_fee = 0.0, just make the value nan
combined['hoa_fee'] = combined['hoa_fee'].replace(0, pd.NA)

# use median imputer for hoa_fee
hoa_fee_median = combined['hoa_fee'].median() 
combined['hoa_fee'].fillna(hoa_fee_median, inplace=True)

# remove values where price_per_sqft < 250 (realistic minimum bound) 
combined.loc[combined['price_per_sqft'] < 250, 'price_per_sqft'] = pd.NA 

# remove values where beds or full_baths or sqft is 0 
remove_cols = ['beds', 'full_baths', 'sqft']
for col in remove_cols:
    combined.loc[combined[col] == 0, col] = pd.NA

# drop the one condop for_rent row 
combined.drop(combined[(combined['status'] == 'FOR_RENT') & (combined['style'] == 'CONDOP')].index)

combined.drop(columns=['half_baths', 
                       'lot_sqft', 
                       'sold_price',
                       'last_sold_date',
                       'neighborhoods',
                       'nearby_schools',], inplace=True)

combined.to_csv('combined.csv', index=False)
combined.head()

/tmp/ipykernel_3507430/3067746824.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  combined['stories'].fillna(stories_median, inplace=True)
/tmp/ipykernel_3507430/3067746824.py:14: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  combined['stories'].fillna(stories_median, inplace=True)
/tmp/ipykernel_3507430/3

,status,style,zip_code,beds,full_baths,sqft,year_built,list_price,new_construction,price_per_sqft,latitude,longitude,stories,hoa_fee
0,FOR_SALE,COOP,10003,NaN,1.0,NaN,1900.0,475000.0,False,NaN,40.724612,-73.988153,11.0,694.0
1,FOR_SALE,COOP,10003,2.0,1.0,NaN,1910.0,865000.0,False,NaN,40.730642,-73.987571,11.0,1280.0
2,FOR_SALE,CONDOS,10003,3.0,2.0,1360.0,1877.0,3700000.0,False,2721.0,40.734846,-73.984337,11.0,2311.0
3,FOR_SALE,COOP,10003,1.0,1.0,NaN,1955.0,2495000.0,False,NaN,40.732628,-73.995850,11.0,3272.0
4,FOR_SALE,COOP,10003,1.0,1.0,NaN,1868.0,1250000.0,False,NaN,40.732857,-73.991730,3.0,2070.0


In [15]:
temp_df = combined.copy() 
temp_df['new_construction'] = temp_df['new_construction'].astype(int)

def calc_distance(latitude: float, longitude: float) -> float: 
    # destination is washington square park
    DESTINATION_LAT = 40.7308
    DESTINATION_LONG = -73.9976

    R = 6371 # earth's radius 

    lat1 = np.radians(latitude) 
    long1 = np.radians(longitude) 

    lat2 = np.radians(DESTINATION_LAT)
    long2 = np.radians(DESTINATION_LONG)

    long_dist = long2 - long1
    lat_dist = lat2 - lat1 

    a = np.sin(lat_dist / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(long_dist / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    
    return R * c # returns distance in km 

combined['park_distance'] = combined.apply(
    lambda row: calc_distance(row['latitude'], row['longitude']),
    axis=1) 

combined.to_csv('combined.csv', index=False)
combined.head()

,status,style,zip_code,beds,full_baths,sqft,year_built,list_price,new_construction,price_per_sqft,latitude,longitude,stories,hoa_fee,park_distance
0,FOR_SALE,COOP,10003,NaN,1.0,NaN,1900.0,475000.0,False,NaN,40.724612,-73.988153,11.0,694.0,1.052214
1,FOR_SALE,COOP,10003,2.0,1.0,NaN,1910.0,865000.0,False,NaN,40.730642,-73.987571,11.0,1280.0,0.845244
2,FOR_SALE,CONDOS,10003,3.0,2.0,1360.0,1877.0,3700000.0,False,2721.0,40.734846,-73.984337,11.0,2311.0,1.204689
3,FOR_SALE,COOP,10003,1.0,1.0,NaN,1955.0,2495000.0,False,NaN,40.732628,-73.995850,11.0,3272.0,0.251117
4,FOR_SALE,COOP,10003,1.0,1.0,NaN,1868.0,1250000.0,False,NaN,40.732857,-73.991730,3.0,2070.0,0.544935


In [16]:
"""
- use knnimputer to impute:
    - beds
    - full_baths
    - year_built 
    - sqft
    - latitude + longitude
"""

combined['new_construction'] = combined['new_construction'].astype(int) 
cols_for_knn = ['beds',
                'full_baths',
                'year_built',
                'sqft',
                'latitude',
                'longitude',
                'zip_code',
                'stories',
                'hoa_fee',
                'new_construction',
                'park_distance'] 

knn_data = combined[cols_for_knn].copy() 


knn_imputer = KNNImputer(
    n_neighbors = 7,
    weights = 'distance',
)

impute_cols = ['beds', 
               'full_baths', 
               'year_built', 
               'sqft', 
               'latitude',
               'longitude']

imputed_data = knn_imputer.fit_transform(knn_data) 
imputed_df = pd.DataFrame(imputed_data, 
                          columns=cols_for_knn,
                          index=combined.index)

for col in impute_cols:
    combined[col] = imputed_df[col]

# round every col except latitude and longitude 
cols_2round = [col for col in impute_cols if col not in ['latitude', 'longitude']]
combined[cols_2round] = combined[cols_2round].round(0)

combined.to_csv('combined2.csv', index=False) # for checking

combined2 = pd.read_csv('combined2.csv')
combined = pd.read_csv('combined.csv')

In [54]:
# drop rows with missing style 
combined2.dropna(subset=['style'], axis=0, inplace=True)

# recalculate distance to park now that lat/long values have been reimputed
combined2['park_distance'] = combined2.apply(
    lambda row: calc_distance(row['latitude'], row['longitude']),
    axis=1) 

print(len(combined2[combined2['park_distance'] == 0])) # prints 0 yay!! 

# drop pending rows; decided not to use
combined2 = combined2[combined2['status'] != 'PENDING']
# also drop price_per_sqft; also decided not to use
combined2.drop('price_per_sqft', axis=1, inplace=True)

combined2.drop(combined2[(combined2['status'] == 'FOR_RENT') & (combined2['style'] == 'CONDOP')].index, inplace=True)
print(combined2['style'].value_counts())

combined2.to_csv('combined2.csv', index=False)
for col in combined2:
    print(col)

0
style
COOP              1077
CONDO              824
CONDOS             766
APARTMENT          743
SINGLE_FAMILY      129
OTHER               69
TOWNHOMES           59
MULTI_FAMILY        40
CONDOP               6
DUPLEX_TRIPLEX       2
Name: count, dtype: int64
status
style
zip_code
beds
full_baths
sqft
year_built
list_price
new_construction
latitude
longitude
stories
hoa_fee
park_distance


In [62]:
# time to one-hot encode everything! 
# columns to encode:
# status (for_sale/sold/for_rent) 
# style (10 diff property types)
# zip_code —– although this is numerical, treat it as categorical 

cols_2encode = ['status', 'style', 'zip_code'] 
final_df = pd.get_dummies(combined2, columns=cols_2encode, drop_first=False) 
final_df['new_construction'] = final_df['new_construction'].astype(bool)

# separate this dataframe into for_sale/sold and for_rent 
sales_df = final_df[(final_df['status_FOR_SALE']) | (final_df['status_SOLD'])]
rentals_df = final_df[final_df['status_FOR_RENT']]

sales_X = sales_df.drop('list_price', axis=1)
rentals_X = rentals_df.drop('list_price', axis=1)

sales_y = sales_df['list_price']
rentals_y = rentals_df['list_price']

sales_df_og = combined2[combined2['status'].isin(['FOR_SALE', 'SOLD'])]
rentals_df_og = combined2[combined2['status'] == 'FOR_RENT']

sales_X_train, sales_X_test, sales_y_train, sales_y_test = train_test_split(sales_X, 
                                                                            sales_y,
                                                                            test_size=0.2,
                                                                            random_state=42,
                                                                            stratify=sales_df_og['style'])

rentals_X_train, rentals_X_test, rentals_y_train, rentals_y_test = train_test_split(rentals_X,
                                                                                    rentals_y, 
                                                                                    test_size=0.2,
                                                                                    random_state=42,
                                                                                    stratify=rentals_df_og['style']) 
# ready to model!

Index([], dtype='int64')


# 5. Model the data
Using selected ML models, experment with your choices and describe your findings. Finish by selecting a Model to continue with


In [88]:
# model of choice: random forest regressor(s) 
# make a separate model for for_sale/sold homes, then one for rentals 

# start with for_sale/sold homes

sales_rf = RandomForestRegressor(
    n_estimators = 500,
    random_state = 42,
    n_jobs = -1,
    max_features = 'sqrt',
)

sales_rf.fit(sales_X_train, sales_y_train)

RandomForestRegressor(max_features='sqrt', n_estimators=500, n_jobs=-1,
                      random_state=42)

In [89]:
sales_y_preds = sales_rf.predict(sales_X_test)
print(f'r2: {r2_score(sales_y_test, sales_y_preds):.2f}')

r2: -0.03


In [96]:
scaler = StandardScaler()
sales_X_train_scaled = scaler.fit_transform(sales_X_train)
sales_X_test_scaled = scaler.transform(sales_X_test)

# Instantiate and fit SVM regressor
svm_reg = SVR(kernel='rbf', C=1.0, epsilon=0.2)
svm_reg.fit(sales_X_train_scaled, sales_y_train)

# Predict
sales_y_preds_svm = svm_reg.predict(sales_X_test_scaled)

# Evaluate
print("SVM R²:", r2_score(sales_y_test, sales_y_preds_svm))
print("SVM MAPE:", mean_absolute_percentage_error(sales_y_test, sales_y_preds_svm))

SVM R²: -0.07292479395858908
SVM MAPE: 1.24346272756126


In [99]:
scaler = StandardScaler()
sales_X_train_scaled = scaler.fit_transform(sales_X_train)
sales_X_test_scaled = scaler.transform(sales_X_test)

# Parameter grid
param_grid = {
    'C': [0.1, 1, 10, 100],
    'epsilon': [0.01, 0.1, 0.2, 0.5],
    'kernel': ['rbf', 'linear', 'poly'],
    'gamma': ['scale', 'auto']
}

# Grid search
svm = SVR()
grid_search = GridSearchCV(svm, param_grid, scoring='r2', cv=3, verbose=2, n_jobs=-1)
grid_search.fit(sales_X_train_scaled, sales_y_train)

print("Best parameters:", grid_search.best_params_)
print("Best R² score (cv):", grid_search.best_score_)

# Test set performance
best_svm = grid_search.best_estimator_
sales_y_preds_best = best_svm.predict(sales_X_test_scaled)
print("Test R²:", r2_score(sales_y_test, sales_y_preds_best))
print("Test MAPE:", mean_absolute_percentage_error(sales_y_test, sales_y_preds_best))


Fitting 3 folds for each of 96 candidates, totalling 288 fits
Best parameters: {'C': 100, 'epsilon': 0.5, 'gamma': 'scale', 'kernel': 'linear'}
Best R² score (cv): -0.02138708132895865
Test R²: 0.002435694706067082
Test MAPE: 1.1575622994117727


# 6. Fine Tune the Model

With the select model descibe the steps taken to acheve the best rusults possiable 


# 7. Present
In a customer faceing Document provide summery of finding and detail approach taken


# 8. Launch the Model System
Define your production run code, This should be self susficent and require only your model pramaters 


In [19]:
def infrence(prams):
    results = m.run(prams)
    return results 